In [1]:
import io, os, uuid, yaml
import numpy as np
import pandas as pd
from azure.storage.blob import BlobServiceClient, BlobClient, ContainerClient

In [2]:
with open("./credentials.yaml","r") as c:
    credentials = yaml.safe_load(c)['blob_storage']

In [3]:
blob_service_client = BlobServiceClient.from_connection_string(credentials['conn_string'])

In [4]:
container_name = "ingredion-data"
container_client = blob_service_client.get_container_client(container_name)

In [20]:
blob_list = container_client.list_blobs()

In [6]:
keys = {'name', "size"}
az_storage_blobs = [{k:v for k,v in b.items() if k in keys} for b in blob_list]

In [7]:
az_storage_blobs

[{'name': 'Brazil_file', 'size': 48885145},
 {'name': 'US_file', 'size': 111936526},
 {'name': 'forecast_ingredion_Brazil', 'size': 12493379},
 {'name': 'forecast_ingredion_US', 'size': 29119569}]

In [8]:
az_storage_blobs[3]['name']

'forecast_ingredion_US'

In [13]:
stream = container_client.download_blob(az_storage_blobs[3]['name'])
result = stream.content_as_text()

In [14]:
data = io.StringIO(result)
df = pd.read_csv(data)

In [15]:
df.shape

(336690, 6)

In [16]:
df.head()

,date,ship_to_party,commercial_name,GBM_forecast,xgb_forecast,target
0,2016-09-01,"DLVD QUALITY LIQUID FEEDS, INC.",GLUCOSE SYRUP INDUSTRIAL,19671.035822,20963.375,8561.35
1,2016-10-01,"DLVD QUALITY LIQUID FEEDS, INC.",GLUCOSE SYRUP INDUSTRIAL,11292.881589,12142.574,-384.39
2,2018-09-01,DUNN PAPER- LADYSMITH,NATIONAL 54-3418,25827.689756,28595.516,64239.11
3,2018-10-01,DUNN PAPER- LADYSMITH,NATIONAL 54-3418,48117.739000,55248.285,31350.02
4,2018-11-01,DUNN PAPER- LADYSMITH,NATIONAL 54-3418,29569.294797,28595.516,33313.79


In [19]:
df.groupby(['date']).sum()/1000000

,GBM_forecast,xgb_forecast,target
date,,,
2016-01-01,167.298497,165.910559,160.518294
2016-02-01,171.379305,171.530981,170.694185
2016-03-01,185.165746,185.411623,167.487145
2016-04-01,177.039662,176.996701,173.441950
2016-05-01,177.610062,175.969136,183.903559
2016-06-01,188.957611,187.871892,169.809625
2016-07-01,177.499726,178.301661,186.605487
2016-08-01,190.380576,189.385387,179.065464
2016-09-01,176.412484,172.479179,162.040115


In [25]:
df.fiscal_year_period.unique()

array([ 1.2016, 10.2016, 11.2016, 12.2016,  2.2016,  3.2016,  4.2016,
        5.2016,  6.2016,  7.2016,  8.2016,  9.2016,  1.2017, 10.2017,
       11.2017, 12.2017,  2.2017,  3.2017,  4.2017,  5.2017,  6.2017,
        7.2017,  8.2017,  9.2017,  1.2018, 10.2018, 11.2018, 12.2018,
        2.2018,  3.2018,  4.2018,  5.2018,  6.2018,  7.2018,  8.2018,
        9.2018,  1.2019, 10.2019, 11.2019, 12.2019,  2.2019,  3.2019,
        4.2019,  5.2019,  6.2019,  7.2019,  8.2019,  9.2019,  1.202 ,
       10.202 , 11.202 , 12.202 ,  2.202 ,  3.202 ,  4.202 ,  5.202 ,
        6.202 ,  7.202 ,  8.202 ,  9.202 ])

In [26]:
df.nunique()

fiscal_year_period                     60
company_name                            1
company_code                            1
Ship_to_party_code                   3155
ship_to_party                        2786
material                              869
commercial_name                       291
sales_qty_total_mt                  16829
3p_sales_qty_total_mt               16675
gross_revenue_usd                  102234
discounts_usd                           1
new_net_revenue                    102253
n3p_net_revenue                     98349
net_corn                            70028
raw_material_other                  19919
utilities                           55393
waste                                   1
repair                              65281
labor                               49807
ohmfg                               40212
supplies_and_packaging              67617
supplies_indirect                   36121
depreciation                        47716
3p_freight_usd                    